# Notebook LLM with RAG
Upload a PDF of notes and ask questions using Retrieval-Augmented Generation (RAG) with a local LLM.

In [ ]:
!pip install pypdf faiss-cpu sentence-transformers transformers ipywidgets

## 1. Upload and Parse PDF

In [ ]:
import PyPDF2
from pathlib import Path
from IPython.display import display
import ipywidgets as widgets

uploader = widgets.FileUpload(accept='.pdf', multiple=False)
display(uploader)

def extract_pdf_text(uploaded):
    if not uploaded.value:
        return ''
    file_info = next(iter(uploaded.value.values()))
    pdf_bytes = file_info['content']
    with open('uploaded_notes.pdf', 'wb') as f:
        f.write(pdf_bytes)
    reader = PyPDF2.PdfReader('uploaded_notes.pdf')
    text = ''
    for page in reader.pages:
        text += page.extract_text() + '\n'
    return text

import time
while not uploader.value:
    time.sleep(1)
notes_text = extract_pdf_text(uploader)
print('Extracted', len(notes_text), 'characters from PDF.')

## 2. Chunk Notes and Create Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

def chunk_text(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i+chunk_size])
        if chunk:
            chunks.append(chunk)
    return chunks

chunks = chunk_text(notes_text)
print(f'Chunked into {len(chunks)} segments.')

embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedder.encode(chunks, show_progress_bar=True)

## 3. Build Vector Store (FAISS)

In [ ]:
import faiss

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype('float32'))
print('Vector store built.')

## 4. Ask Questions (RAG Pipeline)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load local LLM (replace with your Mistral path or model)
llm_model = 'mistralai/Mistral-7B-Instruct-v0.2'  # Example; use your local path if needed
tokenizer = AutoTokenizer.from_pretrained(llm_model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(llm_model, trust_remote_code=True)
llm = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=256)

def retrieve(query, k=3):
    query_emb = embedder.encode([query])
    D, I = index.search(np.array(query_emb).astype('float32'), k)
    return [chunks[i] for i in I[0]]

def rag_answer(query):
    retrieved = retrieve(query)
    context = '\n'.join(retrieved)
    prompt = f'Context:\n{context}\n\nQuestion: {query}\nAnswer:'
    response = llm(prompt)[0]['generated_text'].split('Answer:')[-1].strip()
    return response

# Example usage:
question = 'What is the main topic of the notes?'
print('Q:', question)
print('A:', rag_answer(question))

## 5. Ask Your Own Questions

In [ ]:
import ipywidgets as widgets

qbox = widgets.Text(description='Question:')
out = widgets.Output()

def on_submit(change):
    with out:
        out.clear_output()
        print('Q:', qbox.value)
        print('A:', rag_answer(qbox.value))

qbox.on_submit(on_submit)
display(qbox, out)